In [351]:
import json

import numpy as np
import math
import pandas as pd
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as seabornInstance 

import statsmodels.api as sm
from statsmodels.stats import diagnostic as diag
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

%matplotlib inline

# Load Data

In [352]:
def loadDataframe(fileName, numbRows):
    # load .csv data into a dataframe (df) and make sure only important rows stay
    df = pd.read_csv(fileName)
    df = df[:numbRows]
    #remove column white space
    df.columns = df.columns.str.replace(' ', '')
    #drop url column (it doesn't matter for us)
    if 'url' in df.columns:
        df.drop('url', axis=1, inplace=True)
    # set the dataframe index to the id column
    if 'id' in df.columns:
        df = df.set_index('id')
    # Make sure all data is float
    df = df.astype(float)
    
    return df

# Retrieve Dataframes
train_df = loadDataframe('train.csv', 5000)
predictions_df = loadDataframe('predictions_df.csv', 1000)
regressions_df = pd.read_csv('regressions_df.csv', index_col = 'model')
validation_df = loadDataframe('validation.csv', 1000)
validation_df.drop('shares', axis=1, inplace=True)

# Retrieve regression json
with open('regressions.json') as json_file:
    regressions = json.load(json_file)

# set dataframes to display all columns
pd.options.display.max_columns = None
pd.set_option('display.max_colwidth', -1)

train_df

/Users/diogovalentepcs/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,num_keywords,data_channel_is_lifestyle,data_channel_is_entertainment,data_channel_is_bus,data_channel_is_socmed,data_channel_is_tech,data_channel_is_world,kw_min_min,kw_max_min,kw_avg_min,kw_min_max,kw_max_max,kw_avg_max,kw_min_avg,kw_max_avg,kw_avg_avg,self_reference_min_shares,self_reference_max_shares,self_reference_avg_sharess,weekday_is_monday,weekday_is_tuesday,weekday_is_wednesday,weekday_is_thursday,weekday_is_friday,weekday_is_saturday,weekday_is_sunday,is_weekend,LDA_00,LDA_01,LDA_02,LDA_03,LDA_04,global_subjectivity,global_sentiment_polarity,global_rate_positive_words,global_rate_negative_words,rate_positive_words,rate_negative_words,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1.0,198.0,6.0,47.0,0.914894,1.0,0.964286,1.0,1.0,0.0,2.0,4.744681,4.0,0.0,0.0,0.0,1.0,0.0,0.0,-1.0,321.0,121.000000,100300.0,843300.0,415450.00000,2988.371336,4488.109700,3714.512814,910.0,910.0,910.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.462771,0.050069,0.050348,0.386811,0.050001,0.350000,0.075000,0.021277,0.021277,0.500000,0.500000,0.350000,0.350000,0.350,-0.200000,-0.200000,-0.200000,0.443939,-0.015152,0.056061,0.015152,1170.0
2.0,660.0,7.0,181.0,0.519337,1.0,0.644231,5.0,2.0,1.0,0.0,4.613260,3.0,0.0,0.0,0.0,1.0,0.0,0.0,217.0,735.0,473.666667,1400.0,617900.0,215533.33330,1044.500000,3231.238439,2498.783924,0.0,0.0,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.069391,0.066905,0.726183,0.066743,0.070778,0.578512,0.244008,0.044199,0.005525,0.888889,0.111111,0.385511,0.136364,1.000,-0.400000,-0.400000,-0.400000,0.000000,0.000000,0.500000,0.000000,6265.0
3.0,552.0,9.0,862.0,0.465089,1.0,0.635478,16.0,0.0,1.0,0.0,4.725058,7.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,2000.0,377.333333,0.0,843300.0,146642.85710,0.000000,6417.240000,2543.161561,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.028869,0.028572,0.742863,0.170934,0.028762,0.510478,0.195651,0.059165,0.023202,0.718310,0.281690,0.465324,0.050000,1.000,-0.264444,-0.750000,-0.100000,0.000000,0.000000,0.500000,0.000000,121.0
4.0,559.0,10.0,1015.0,0.447503,1.0,0.636986,12.0,2.0,1.0,0.0,4.434483,10.0,1.0,0.0,0.0,0.0,0.0,0.0,4.0,586.0,235.555556,0.0,690400.0,102570.00000,0.000000,3776.977855,2572.800545,1100.0,1100.0,1100.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.020216,0.020014,0.020007,0.020318,0.919446,0.518714,0.214039,0.066995,0.015764,0.809524,0.190476,0.390143,0.100000,1.000,-0.277083,-0.800000,-0.008333,0.000000,0.000000,0.500000,0.000000,841.0
5.0,573.0,8.0,129.0,0.666667,1.0,0.790123,3.0,0.0,1.0,0.0,4.674419,7.0,1.0,0.0,0.0,0.0,0.0,0.0,4.0,469.0,259.400000,0.0,690400.0,117200.00000,0.000000,3335.052308,2029.340699,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.028901,0.028580,0.029893,0.030580,0.882047,0.327778,-0.165741,0.015504,0.038760,0.285714,0.714286,0.237500,0.100000,0.375,-0.393333,-0.700000,-0.166667,0.000000,0.000000,0.500000,0.000000,376.0
6.0,149.0,9.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,2.0,0.000000,7.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,952.0,154.428571,21900.0,843300.0,363285.71430,3098.573478,9494.000000,5069.770031,0.0,0.0,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.028572,0.544207,0.028572,0.370077,0.028572,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,1.000000,0.500000,0.500000,0.500000,16034.0
7.0,702.0,8.0,234.0,0.595652,1.0,0.746377,12.0,4.0,1.0,0.0,4.594017,6.0,0.0,0.0,0.0,1.0,0.0,0.0,217.0,11900.0,2337.333333,8000.0,69100.0,30983.33333,2167.275862,11900.000000,4744.522336,2100.0,5700.0,3900.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.436433,0.033365,0.033470,0.463240,0.033492,0.650317,0.280357,0.025641,0.0085

# Define combinations

In [353]:

# Define the combination group
Grupo0 : ['n_unique_tokens']
Grupo0 : ['num_hrefs'
Grupo0 : ['num_imgs'
Grupo0 : ['num_videos'
Grupo0 : ['n_unique_tokens']
Grupo0 : ['num_hrefs']
Grupo1 : ['num_imgs']
Grupo2 : ['num_videos']
Grupo3 : ['num_keywords']
Grupo4 : ['data_channel_is_entertainment']
Grupo5 : ['data_channel_is_bus']
Grupo6 : ['data_channel_is_world']
Grupo7 : [Normalizada('kw_avg_max')]
Grupo8 : [Normalizada('kw_min_avg')]
Grupo9 : [Normalizada('kw_avg_avg')]
Grupo10 : [Normalizada('self_reference_avg_sharess');ln('self_reference_avg_sharess'+1)]
Grupo11 : ['weekday_is_tuesday']
Grupo12 : ['weekday_is_wednesday']
Grupo13 : ['weekday_is_thursday']
Grupo14 : ['is_weekend']
Grupo15 : ['LDA_01']
Grupo16 : ['avg_negative_polarity']
Grupo17 : ['max_negative_polarity']
Grupo18 : ['title_subjectivity']
Grupo19 : ['abs_title_subjectivity']
Grupo19 : ['n_unique_tokens']


group = [group0, group1, group2, group3, group4, group5, group6, group7, group8, group9, group10,group11, group12, group13, group14,group15]

combinations = []
# Create the different combinations
for i in group0:
    for j in group1:
        for k in group2:
            for l in group3:
                for m in group4:
                    for n in group5:
                        for o in group6:
                            for p in group7:
                                for q in group8:
                                    for r in group9:
                                        for s in group10:
                                            for t in group11:
                                                for u in group12:
                                                    for v in group13:
                                                        for w in group14:
                                                            for x in group15:
                                                                combinations.append([i,j,k,l,m,n,o,p,q, r, s, t, u, v, w, x])
                                        


# The numb of combinations has to be the multplication of the number of variables of each group
numbCombinations = 1
for i in range(0, len(group)):
    numbCombinations = numbCombinations * len(group[i])
# Check its okay
print('Number of combinations created: ',len(combinations))
print('Condition is met:', len(combinations) == numbCombinations)


Number of combinations created:  768
Condition is met: True


In [354]:
# Save the combination groups
def saveCombinationGroup(varGroups):
    with open('Combination_Study/allCombinationsGroups.json') as json_file:
        combinationGroupsList = json.load(json_file)
    # Set the new combination number
    combinationGroupNumber = len(combinationGroupsList)
    # Create the new combination dictionary
    combinationGroup = {'combination' : combinationGroupNumber}
    for i in range(0, len(varGroups)):
        combinationGroup['group{}'.format(i)] = varGroups[i]
    # Add combination do combinationList
    combinationGroupsList.append(combinationGroup)
    # Save combinationsList and new combination
    with open('Combination_Study/allCombinationsGroups.json'.format(combinationGroupNumber), 'w') as outfile:
            json.dump(combinationGroupsList, outfile)
    with open('Combination_Study/combinationGroup_{}.json'.format(combinationGroupNumber), 'w') as outfile:
            json.dump(combinationGroup, outfile)
    
    return combinationGroup, combinationGroupNumber


# Call the saveCombinationGroup in json dictionary
combinationGroup, combinationGroupNumber = saveCombinationGroup(group)
print(combinationGroupNumber)
group

21


[['n_tokens_title'],
 ['n_unique_tokens', 'n_non_stop_words'],
 ['num_imgs'],
 ['data_channel_is_entertainment'],
 ['kw_max_min', 'kw_avg_min'],
 ['kw_max_max', ' kw_min_min'],
 ['kw_avg_max'],
 ['kw_min_avg'],
 ['kw_max_avg', 'kw_avg_avg'],
 ['self_reference_min_shares',
  'self_reference_max_shares',
  'self_reference_avg_sharess'],
 ['weekday_is_saturday', 'is_weekend'],
 ['LDA_01'],
 ['global_subjectivity', 'avg_positive_polarity'],
 ['min_negative_polarity', 'avg_negative_polarity'],
 ['title_sentiment_polarity', 'abs_title_sentiment_polarity'],
 ['abs_title_subjectivity']]

# Tranformations

In [355]:
# Transform variables functions
def logVar(var):
    var += 1
    var = np.log(var)
    return var

def poweredTo(var, power):
    var = var**power
    return var

def multiply(var, factor):
    var = var*factor
    return var


# Do transformations
transf_train_df = train_df.copy()
transf_validation_df = validation_df.copy()

# Dependent Variable transformations (will require inverse transformation)
#transf_train_df = transf_train_df.apply(lambda x: poweredTo(x,(1/3)) if x.name == 'shares' else x) # cube root of train shares

# Independent Variable transformations
#transf_train_df = transf_train_df.apply(lambda x: logVar(x) if x.name == 'self_reference_avg_sharess' else x)
#transf_train_df = transf_train_df.apply(lambda x: poweredTo(x,3) if x.name == 'global_subjectivity' else x)
#transf_validation_df = transf_validation_df.apply(lambda x: logVar(x) if x.name == 'self_reference_avg_sharess' else x)
#transf_validation_df = transf_validation_df.apply(lambda x: poweredTo(x,3) if x.name == 'global_subjectivity' else x)

# Add Components
#transf_train_df['componente_Guida'] = 0.958 * transf_train_df['data_channel_is_world'] + 0.958*transf_train_df['LDA_02']
#transf_validation_df['componente_Guida'] = 0.958 * transf_validation_df['data_channel_is_world'] + 0.958*transf_validation_df['LDA_02']

transf_validation_df

,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,num_keywords,data_channel_is_lifestyle,data_channel_is_entertainment,data_channel_is_bus,data_channel_is_socmed,data_channel_is_tech,data_channel_is_world,kw_min_min,kw_max_min,kw_avg_min,kw_min_max,kw_max_max,kw_avg_max,kw_min_avg,kw_max_avg,kw_avg_avg,self_reference_min_shares,self_reference_max_shares,self_reference_avg_sharess,weekday_is_monday,weekday_is_tuesday,weekday_is_wednesday,weekday_is_thursday,weekday_is_friday,weekday_is_saturday,weekday_is_sunday,is_weekend,LDA_00,LDA_01,LDA_02,LDA_03,LDA_04,global_subjectivity,global_sentiment_polarity,global_rate_positive_words,global_rate_negative_words,rate_positive_words,rate_negative_words,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5001,42.0,11.0,611.0,0.500000,1.0,0.591940,44.0,2.0,0.0,0.0,5.135843,5.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,0.0,-1.000000,0.0,843300.0,325140.000000,0.000000,3409.863989,2005.406182,916.0,2000.0,1458.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.040002,0.040002,0.839992,0.040001,0.040003,0.454508,0.041911,0.032733,0.021277,0.606061,0.393939,0.295942,0.100000,0.800000,-0.299038,-0.750000,-0.187500,0.233333,-0.250000,0.266667,0.250000
5002,90.0,12.0,1302.0,0.459224,1.0,0.639949,28.0,5.0,5.0,8.0,4.570661,9.0,0.0,0.0,0.0,0.0,1.0,0.0,-1.0,715.0,88.500000,0.0,843300.0,353388.888900,0.000000,4093.365854,2899.145503,715.0,6600.0,2965.250000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.022257,0.022298,0.022231,0.240906,0.692307,0.462390,0.154286,0.046083,0.013825,0.769231,0.230769,0.351287,0.062500,1.000000,-0.192460,-0.400000,-0.050000,0.677273,0.468182,0.177273,0.468182
5003,251.0,9.0,857.0,0.487242,1.0,0.642586,12.0,4.0,1.0,0.0,5.122520,8.0,0.0,0.0,0.0,1.0,0.0,0.0,-1.0,519.0,233.400000,0.0,843300.0,208662.500000,0.000000,3889.499495,1961.942199,1000.0,4400.0,2633.333333,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.026493,0.025062,0.770247,0.025260,0.152937,0.477285,0.156786,0.043174,0.011669,0.787234,0.212766,0.317217,0.033333,0.600000,-0.141667,-0.250000,-0.100000,0.000000,0.000000,0.500000,0.000000
5004,674.0,11.0,43.0,0.930233,1.0,0.966667,2.0,1.0,0.0,1.0,5.209302,8.0,0.0,0.0,0.0,0.0,0.0,0.0,217.0,482.0,352.833333,0.0,617900.0,242000.000000,0.000000,4816.633851,2516.680847,7300.0,7300.0,7300.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.025034,0.025058,0.155423,0.769476,0.025009,0.625000,0.131250,0.069767,0.046512,0.600000,0.400000,0.375000,0.250000,0.500000,-0.800000,-1.000000,-0.600000,0.000000,0.000000,0.500000,0.000000
5005,402.0,10.0,394.0,0.558673,1.0,0.764706,8.0,1.0,1.0,0.0,4.421320,10.0,1.0,0.0,0.0,0.0,0.0,0.0,-1.0,806.0,398.142857,0.0,843300.0,129400.000000,0.000000,5385.045000,2637.775584,0.0,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.570893,0.020006,0.020002,0.270556,0.118543,0.540476,0.212500,0.043147,0.022843,0.653846,0.346154,0.436765,0.050000,1.000000,-0.209414,-0.500000,-0.025000,1.000000,0.000000,0.500000,0.000000
5006,199.0,6.0,148.0,0.659864,1.0,0.829268,2.0,2.0,2.0,0.0,4.648649,9.0,0.0,0.0,0.0,0.0,1.0,0.0,-1.0,834.0,254.833333,0.0,843300.0,196244.444400,0.000000,4751.653846,2247.084385,7400.0,7600.0,7500.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.022898,0.022232,0.140509,0.130939,0.683422,0.497802,0.157193,0.060811,0.006757,0.900000,0.100000,0.232612,0.100000,0.400000,-0.050000,-0.050000,-0.050000,0.000000,0.000000,0.500000,0.000000
5007,664.0,11.0,457.0,0.471264,1.0,0.591241,15.0,4.0,12.0,0.0,4.424508,10.0,0.0,0.0,0.0,0.0,1.0,0.0,217.0,596.0,324.400000,9600.0,617900.0,99340.000000,2108.081081,4182.905844,3121.717862,2000.0,6300.0,3433.333333,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.020110,0.020023,0.020015,0.020085,0.919766,0.501566,0.158081,0.039387,0.010941,0.

# Train Single Regression

In [356]:
# Defining Dependent Variable (Y) and Independent Variables (x)
def createXY(train_data): #train_data as the train dataframe 
    
    X = train_data.drop('shares', axis = 1)
    Y = train_data['shares'] 
    return X, Y
X, Y = createXY(transf_train_df)
X

,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,num_keywords,data_channel_is_lifestyle,data_channel_is_entertainment,data_channel_is_bus,data_channel_is_socmed,data_channel_is_tech,data_channel_is_world,kw_min_min,kw_max_min,kw_avg_min,kw_min_max,kw_max_max,kw_avg_max,kw_min_avg,kw_max_avg,kw_avg_avg,self_reference_min_shares,self_reference_max_shares,self_reference_avg_sharess,weekday_is_monday,weekday_is_tuesday,weekday_is_wednesday,weekday_is_thursday,weekday_is_friday,weekday_is_saturday,weekday_is_sunday,is_weekend,LDA_00,LDA_01,LDA_02,LDA_03,LDA_04,global_subjectivity,global_sentiment_polarity,global_rate_positive_words,global_rate_negative_words,rate_positive_words,rate_negative_words,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1.0,198.0,6.0,47.0,0.914894,1.0,0.964286,1.0,1.0,0.0,2.0,4.744681,4.0,0.0,0.0,0.0,1.0,0.0,0.0,-1.0,321.0,121.000000,100300.0,843300.0,415450.00000,2988.371336,4488.109700,3714.512814,910.0,910.0,910.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.462771,0.050069,0.050348,0.386811,0.050001,0.350000,0.075000,0.021277,0.021277,0.500000,0.500000,0.350000,0.350000,0.350,-0.200000,-0.200000,-0.200000,0.443939,-0.015152,0.056061,0.015152
2.0,660.0,7.0,181.0,0.519337,1.0,0.644231,5.0,2.0,1.0,0.0,4.613260,3.0,0.0,0.0,0.0,1.0,0.0,0.0,217.0,735.0,473.666667,1400.0,617900.0,215533.33330,1044.500000,3231.238439,2498.783924,0.0,0.0,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.069391,0.066905,0.726183,0.066743,0.070778,0.578512,0.244008,0.044199,0.005525,0.888889,0.111111,0.385511,0.136364,1.000,-0.400000,-0.400000,-0.400000,0.000000,0.000000,0.500000,0.000000
3.0,552.0,9.0,862.0,0.465089,1.0,0.635478,16.0,0.0,1.0,0.0,4.725058,7.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,2000.0,377.333333,0.0,843300.0,146642.85710,0.000000,6417.240000,2543.161561,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.028869,0.028572,0.742863,0.170934,0.028762,0.510478,0.195651,0.059165,0.023202,0.718310,0.281690,0.465324,0.050000,1.000,-0.264444,-0.750000,-0.100000,0.000000,0.000000,0.500000,0.000000
4.0,559.0,10.0,1015.0,0.447503,1.0,0.636986,12.0,2.0,1.0,0.0,4.434483,10.0,1.0,0.0,0.0,0.0,0.0,0.0,4.0,586.0,235.555556,0.0,690400.0,102570.00000,0.000000,3776.977855,2572.800545,1100.0,1100.0,1100.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.020216,0.020014,0.020007,0.020318,0.919446,0.518714,0.214039,0.066995,0.015764,0.809524,0.190476,0.390143,0.100000,1.000,-0.277083,-0.800000,-0.008333,0.000000,0.000000,0.500000,0.000000
5.0,573.0,8.0,129.0,0.666667,1.0,0.790123,3.0,0.0,1.0,0.0,4.674419,7.0,1.0,0.0,0.0,0.0,0.0,0.0,4.0,469.0,259.400000,0.0,690400.0,117200.00000,0.000000,3335.052308,2029.340699,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.028901,0.028580,0.029893,0.030580,0.882047,0.327778,-0.165741,0.015504,0.038760,0.285714,0.714286,0.237500,0.100000,0.375,-0.393333,-0.700000,-0.166667,0.000000,0.000000,0.500000,0.000000
6.0,149.0,9.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,2.0,0.000000,7.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,952.0,154.428571,21900.0,843300.0,363285.71430,3098.573478,9494.000000,5069.770031,0.0,0.0,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.028572,0.544207,0.028572,0.370077,0.028572,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,1.000000,0.500000,0.500000,0.500000
7.0,702.0,8.0,234.0,0.595652,1.0,0.746377,12.0,4.0,1.0,0.0,4.594017,6.0,0.0,0.0,0.0,1.0,0.0,0.0,217.0,11900.0,2337.333333,8000.0,69100.0,30983.33333,2167.275862,11900.000000,4744.522336,2100.0,5700.0,3900.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.436433,0.033365,0.033470,0.463240,0.033492,0.650317,0.280357,0.025641,0.008547,0.750000,0.250000,0.500000,0.500000,0.500,-0.

In [357]:
# Train the regression
def trainRegression(X, Y):
    
    # Add constant to X to creat linear model constant
    X = sm.add_constant(X)  
    
    # Split X and Y into train and test parts 
    # Using 70% of the data to train the model and 30% to test it
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.30)


    # Train a linear regression using Ordinary Least Squares
    regression = sm.OLS(y_train, x_train)
    
    # Fit the regression model
    regression_model = regression.fit()
    return regression_model, x_train, x_test, y_train, y_test

X,Y = createXY(transf_train_df)
regression_model, x_train, x_test, y_train, y_test = trainRegression(X,Y)

In [358]:
# Retrieve model dictionary with coeff
def retrieveModelCoeff(regression_model):
    model_coeff = {}
    for key in regression_model.params.keys(): #loops through all variables
        model_coeff[key] = regression_model.params[key]

    for column in train_df.drop('shares', axis = 1):
        if column not in model_coeff.keys():
            model_coeff[column] = 0
    return model_coeff


In [359]:
# Create a prediction (x_predict = dataframe with idependent variable used to predict y_predict values; linear model in dictionary form)
def modelPredict(x_predict, linear_model):
    y = 0
    y_predict = []
    for row in x_predict.index:
        y = linear_model['const']
        for column in x_predict.columns:
            y += x_predict[column][row]*linear_model[column]
        y_predict.append(y)
    # Create a series from the list
    y_predict = pd.Series(y_predict, index = x_predict.index)
    y_predict.index.name = 'id'
    return y_predict



# Model Analysis

## Retrieving Model Stats

In [360]:
# Stats
def retrieveStats(y_real, y_pred, x_used_to_pred):
    real_avg_shares = y_real.mean()
    r2_pred = 0
    r2_real = 0
    for index, value in y_real.iteritems():
        r2_pred += (real_avg_shares - y_pred[index])**2 #y_pred[index] = previsão para o indice atual
        r2_real += (real_avg_shares - y_real[index])**2 #y_real[index] = shares reais para o indice atual
    r2 = r2_pred/ r2_real
    n_observations = len(y_real)
    n_variables_used = len(x_used_to_pred.columns)
    adj_r2 = 1 - (1-r2)*(n_observations - 1)/(n_observations - 1 - n_variables_used)
    
    mae = metrics.mean_absolute_error(y_real, y_pred)
    rmse = math.sqrt(metrics.mean_squared_error(y_real, y_pred))
    
    try:
        rmsle = math.sqrt(metrics.mean_squared_log_error(y_real, y_pred))
    except:
        rmsle = -1
    
    return r2, adj_r2, mae, rmse, rmsle, n_observations, n_variables_used



# Train Model via Iteration

In [361]:
# Model training function (X = independent variables data, Y = independent variable data)
def createModel(X, Y, train_data, x_validation):
        
    # Create models' dataframe with all variables but empty
    models = train_data.drop('shares', axis = 1).iloc[0:0]

    # Add column for the constant (on first column)
    models.insert(0, 'const', [], True)
    
    # Create a dataframe for pvalues
    pvalues_df = models.copy()
    
    # Iteration for regression creation
    for i in range(0,100):
        
        # Train Single Regression
        regression_model,  x_train, x_test, y_train, y_test  = trainRegression(X, Y)
        
        # Retrieve Coefficient Dicitionary
        model_coeff = retrieveModelCoeff(regression_model)

        # Add the new model to models
        models = models.append(model_coeff, ignore_index=True)
        
        # Add new model pvalues to pvalues_df
        pvalues_df = pvalues_df.append(regression_model.pvalues, ignore_index=True)
    
        
        
    # Create the final model dictionary from the averages of the iterations
    final_model = {}
    for column in models.columns:
        final_model[column] = models[column].mean()
        # change null coefficients to 0
        if math.isnan(final_model[column]):
            final_model[column] = 0
    
    # Predict results for stats with final model
    y_train_predicted = modelPredict(X, final_model)
    
    ######################################################################################################
    ######################### Delete if no transformations on shares are done ############################    
    ######################################################################################################
    
    # Do the inverse transformation for both y_train_predicted and y_train_real (= Y)
    #y_train_predicted = y_train_predicted.apply(lambda x: poweredTo(x,3))
    #Y = Y.apply(lambda x: poweredTo(x,3))
    
    ######################################################################################################
    ############################# ------------------------------------- ##################################
    ######################################################################################################
    
    # Retrive Stats for final model
    r2, adj_r2, mae, rmse, rmsle, n_observations, n_variables_used = retrieveStats(Y, y_train_predicted, X)
    
    # Add Stats to the model
    final_model['r2'] = r2
    final_model['adj_r2'] = adj_r2
    final_model['mae'] = mae
    final_model['rmse'] = rmse
    final_model['rmsle'] = rmsle
    final_model['n_variables_used'] = n_variables_used
    #final_model['f_pvalue'] = f_pvalue
    
    # Add description
    final_model['variables_used'] = ';'.join(X.columns)
    
    # Create final pvalues dictionary from the averages of the iterations
    final_pvalues = {}
    for column in pvalues_df.columns:
        final_pvalues[column] = pvalues_df[column].mean()
        # change null coefficients to 0
        if math.isnan(final_pvalues[column]):
            final_pvalues[column] = 0    
    
    # Predict results for stats with final model
    y_validation_predicted = modelPredict(x_validation, final_model)
    
    
    ######################################################################################################
    ######################### Delete if no transformations on shares are done ############################    
    ######################################################################################################
    
    # Do the inverse transformation for y_validation_predicted
    #y_validation_predicted = y_validation_predicted.apply(lambda x: poweredTo(x,3))
    
    ######################################################################################################
    ############################# ------------------------------------- ##################################
    ######################################################################################################
    
    
    return final_model, final_pvalues, y_validation_predicted, models, pvalues_df

# Defining Dependent Variable (Y) and Independent Variables (x)
#X, Y = createXY(train_df)
#final_model, final_pvalues, final_pred, models_df, pvalues_df = createModel(X, Y, train_df)
#models_df

In [362]:
#Visualize final_model
#final_model_df = pd.DataFrame(data=final_model, index = [0])
#final_model_df

# Training Regression Models for each Combination

In [363]:
# Create dataframe for the models created with each combination
regressionModels_df = pd.read_csv('Combination_Study/base_df.csv', index_col = 'model')

# Create list for the models created with each combination
regressionModels_list = []
preidcition_list = []

for combination in combinations:
    
    # Create X and Y from data without outliers
    X, Y = createXY(transf_train_df)

    
    # Drop independet variables not part of the combination
    for var in X.columns:
        if var not in combination:
            X.drop(var, axis=1, inplace=True)
    
    
    
    final_model, final_pvalues, final_pred, models_df, pvalues_df = createModel(X, Y, transf_train_df, transf_validation_df)
    
    # Add prediction to list
    preidcition_list.append(final_pred)
    
    # Add final model of combination to regressions list
    regressionModels_list.append(final_model)
    
    # Add final model of combination to regression models dataframe
    regressionModels_df = regressionModels_df.append(final_model, ignore_index=True)
    regressionModels_df.index.rename('model', inplace = True)

    #print(len(regressionModels_list))
# Create .json file with list for the models created with each combination of combination group
#with open('Combination_Study/combinationGroup_{}_models.json'.format(combinationNummber), 'w') as outfile:
 #   json.dump(regressionModels_list, outfile)
# Create .csv file dataframe for the models created with each combination of combination group
#regressionModels_df.to_csv('Combination_Study/combinationGroup_{}_models.csv'.format(combinationNummber))

regressionModels_df

,const,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,num_keywords,data_channel_is_lifestyle,data_channel_is_entertainment,data_channel_is_bus,data_channel_is_socmed,data_channel_is_tech,data_channel_is_world,kw_min_min,kw_max_min,kw_avg_min,kw_min_max,kw_max_max,kw_avg_max,kw_min_avg,kw_max_avg,kw_avg_avg,self_reference_min_shares,self_reference_max_shares,self_reference_avg_sharess,weekday_is_monday,weekday_is_tuesday,weekday_is_wednesday,weekday_is_thursday,weekday_is_friday,weekday_is_saturday,weekday_is_sunday,is_weekend,LDA_00,LDA_01,LDA_02,LDA_03,LDA_04,global_subjectivity,global_sentiment_polarity,global_rate_positive_words,global_rate_negative_words,rate_positive_words,rate_negative_words,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,adj_r2,f_pvalue,mae,mse,r2,rmse,otherDescription,trasnformedVariables,uesdComponents,usedVariables,n_variables_used,rmsle,variables_used
model,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,36.763686,0.0,66.094438,0.0,-5.336080,0.000000,0.0,0.0,0.0,48.572955,0.0,0.0,0.0,0.0,-475.502797,0.0,0.0,0.0,0.0,0.0,0.005834,0.000000,0.0,-0.001334,0.003308,0.124958,0.162551,0.000000,0.015005,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,1009.378902,0.0,0.000000,0.0,-398.677340,0.0,0.0,0.0,2637.931641,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,-265.228173,0.0,0.0,1059.587922,546.122295,0.000000,0.024290,NaN,3087.798690,NaN,0.027413,8104.337592,NaN,NaN,NaN,NaN,16.0,-1.000000,n_tokens_title;n_unique_tokens;num_imgs;data_channel_is_entertainment;kw_max_min;kw_max_max;kw_avg_max;kw_min_avg;kw_max_avg;self_reference_min_shares;weekday_is_saturday;LDA_01;global_subjectivity;min_negative_polarity;title_sentiment_polarity;abs_title_subjectivity
1,-219.702274,0.0,67.431203,0.0,25.899528,0.000000,0.0,0.0,0.0,47.595848,0.0,0.0,0.0,0.0,-495.077740,0.0,0.0,0.0,0.0,0.0,-0.003284,0.000000,0.0,-0.001231,0.003166,0.128948,0.153011,0.000000,0.016463,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,981.513725,0.0,0.000000,0.0,-456.319133,0.0,0.0,0.0,2408.906235,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,-191.866295,0.0,0.0,0.000000,1066.814387,1807.460178,0.024874,NaN,3082.403668,NaN,0.027995,8102.838078,NaN,NaN,NaN,NaN,16.0,1.089090,n_tokens_title;n_unique_tokens;num_imgs;data_channel_is_entertainment;kw_max_min;kw_max_max;kw_avg_max;kw_min_avg;kw_max_avg;self_reference_min_shares;weekday_is_saturday;LDA_01;global_subjectivity;min_negative_polarity;abs_title_subjectivity;abs_title_sentiment_polarity
2,-92.795838,0.0,68.879153,0.0,49.227097,0.000000,0.0,0.0,0.0,48.455432,0.0,0.0,0.0,0.0,-510.895628,0.0,0.0,0.0,0.0,0.0,0.005242,0.000000,0.0,-0.001295,0.003128,0.121652,0.161828,0.000000,0.016514,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,1058.102592,0.0,0.000000,0.0,-426.610272,0.0,0.0,0.0,2307.627728,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,-1413.955867,0.000000,0.0,0.0,1090.035512,551.867822,0.000000,0.028427,NaN,3085.316018,NaN,0.031537,8115.411721,NaN,NaN,NaN,NaN,16.0,1.089390,n_tokens_title;n_unique_tokens;num_imgs;data_channel_is_entertainment;kw_max_min;kw_max_max;kw_avg_max;kw_min_avg;kw_max_avg;self_reference_min_shares;weekday_is_saturday;LDA_01;global_subjectivity;avg_negative_polarity;title_sentiment_polarity;abs_title_subjectivity
3,-224.167044,0.0,66.198748,0.0,75.819738,0.000000,0.0,0.0,0.0,49.764437,0.0,0.0,0.0,0.0,-500.665667,0.0,0.0,0.0,0.0,0.0,0.004955,0.000000,0.0,-0.001309,0.003159,0.127087,0.163358,0.000000,0.015049,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,1011.019334,0.0,0.000000,0.0,-410.076400,0.0,0.0,0.0,2027.669464,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,-1050.799928,0.000000,0.0,0.0,0.000000,1017.780548,1732.269383,0.034054,NaN,3087.453241,NaN,0.037145,8130.522346,NaN,NaN,NaN,NaN,16.0,1.088116,n_to

In [365]:
min_rmse = 100000000
bestIndex = 0
for index, row in regressionModels_df.iterrows():
    if row['r2'] > 0.03:
        if row['rmse'] < min_rmse:
            min_rmse = row['rmse']
            bestIndex = index
print(bestIndex )

#delete all rows in regressions except for best
bestModel = regressionModels_df.copy()
for index, row in bestModel.iterrows():
    if index != bestIndex :
        bestModel = bestModel.drop(index)
bestModel = bestModel.reset_index(drop=True)

bestModel

267


,const,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,num_keywords,data_channel_is_lifestyle,data_channel_is_entertainment,data_channel_is_bus,data_channel_is_socmed,data_channel_is_tech,data_channel_is_world,kw_min_min,kw_max_min,kw_avg_min,kw_min_max,kw_max_max,kw_avg_max,kw_min_avg,kw_max_avg,kw_avg_avg,self_reference_min_shares,self_reference_max_shares,self_reference_avg_sharess,weekday_is_monday,weekday_is_tuesday,weekday_is_wednesday,weekday_is_thursday,weekday_is_friday,weekday_is_saturday,weekday_is_sunday,is_weekend,LDA_00,LDA_01,LDA_02,LDA_03,LDA_04,global_subjectivity,global_sentiment_polarity,global_rate_positive_words,global_rate_negative_words,rate_positive_words,rate_negative_words,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,adj_r2,f_pvalue,mae,mse,r2,rmse,otherDescription,trasnformedVariables,uesdComponents,usedVariables,n_variables_used,rmsle,variables_used
0,-1678.618802,0.0,79.221552,0.0,33.42907,0.0,0.0,0.0,0.0,32.670803,0.0,0.0,0.0,0.0,-484.052553,0.0,0.0,0.0,0.0,0.0,0.0,-0.531708,0.0,-0.001286,-0.000267,-0.285591,0.0,1.273228,0.0,0.002435,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,725.678106,0.0,-716.664241,0.0,0.0,0.0,1845.810586,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-962.977941,0.0,0.0,0.0,0.0,1074.876432,1680.911942,0.039121,NaN,3008.183559,NaN,0.042196,8046.93687,NaN,NaN,NaN,NaN,16.0,1.059127,n_tokens_title;n_unique_tokens;num_imgs;data_channel_is_entertainment;kw_avg_min;kw_max_max;kw_avg_max;kw_min_avg;kw_avg_avg;self_reference_max_shares;is_weekend;LDA_01;global_subjectivity;avg_negative_polarity;abs_title_subjectivity;abs_title_sentiment_polarity


In [366]:

best_prediction = preidcition_list[bestIndex]
best_prediction

id
5001    2401.761737
5002    3932.235427
5003    1864.323423
5004    3842.423709
5005    2914.786133
5006    1979.489084
5007    3690.771626
5008    3466.165086
5009    3390.749457
5010    1332.658138
5011    2445.965174
5012    6751.097140
5013    2373.353175
5014    2005.587168
5015    4584.038523
5016    2479.210454
5017    2767.188872
5018    3844.484184
5019    2247.261388
5020    2928.238300
5021    2417.350402
5022    4197.451575
5023    4469.586831
5024    2488.715294
5025    2829.616427
5026    2358.230199
5027    2019.465603
5028    2724.396667
5029    2598.281610
5030    2994.243209
           ...     
5971    5150.577747
5972    3237.948240
5973    2008.511977
5974    4242.516740
5975    4704.893633
5976    2034.085003
5977    3104.531555
5978    4094.364202
5979    3986.374013
5980    2497.689787
5981    2508.551351
5982    3817.098329
5983    1501.743322
5984    3170.305919
5985    2782.138048
5986    1661.585370
5987    2970.323210
5988    3429.819314
5989    1964.4578

In [367]:
bestModels = pd.read_csv('combinationModels.csv', index_col = 'model')
bestModels = bestModels.append(bestModel, ignore_index = True)
bestModels.index.rename('model', inplace = True)
bestModels.to_csv('combinationModels.csv')
bestModels

/Users/diogovalentepcs/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


,LDA_00,LDA_01,LDA_02,LDA_03,LDA_04,abs_title_sentiment_polarity,abs_title_subjectivity,adj_r2,average_token_length,avg_negative_polarity,avg_positive_polarity,componente_Guida,const,data_channel_is_bus,data_channel_is_entertainment,data_channel_is_lifestyle,data_channel_is_socmed,data_channel_is_tech,data_channel_is_world,f_pvalue,global_rate_negative_words,global_rate_positive_words,global_sentiment_polarity,global_subjectivity,is_weekend,kw_avg_avg,kw_avg_max,kw_avg_min,kw_max_avg,kw_max_max,kw_max_min,kw_min_avg,kw_min_max,kw_min_min,mae,max_negative_polarity,max_positive_polarity,min_negative_polarity,min_positive_polarity,mse,n_non_stop_unique_tokens,n_non_stop_words,n_tokens_content,n_tokens_title,n_unique_tokens,n_variables_used,num_hrefs,num_imgs,num_keywords,num_self_hrefs,num_videos,otherDescription,r2,rate_negative_words,rate_positive_words,rmse,rmsle,self_reference_avg_sharess,self_reference_max_shares,self_reference_min_shares,timedelta,title_sentiment_polarity,title_subjectivity,transformations,trasnformedVariables,uesdComponents,usedVariables,variables_used,weekday_is_friday,weekday_is_monday,weekday_is_saturday,weekday_is_sunday,weekday_is_thursday,weekday_is_tuesday,weekday_is_wednesday
model,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.000000,0.000000,0.000000,0.791144,0.000000,0.711531,0.000000,3.355184e-02,0.000000,0.000000,0.000000,-0.558483,8.680919,0.000000,-1.296220,0.000000,0.000000,0.000000,0.000000,NaN,0.00000,0.0,0.000000,3.360135,1.303315,0.000694,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,2.476041e+03,0.0,0.000000,0.0,0.0,NaN,0.0,0.000000,0.000000,0.000000,0.00000,10.0,0.0,0.023678,0.103053,0.0,0.000000,NaN,3.548512e-02,0.000000,0.000000,8.167093e+03,0.899273,0.115163,0.000000,0.0,0.0,0.0,0.0,shares^(1/3); ln(self_reference_avg_sharess + 1); (global_sbjectivity)^3,NaN,NaN,NaN,num_imgs;num_keywords;data_channel_is_entertainment;kw_avg_avg;self_reference_avg_sharess;is_weekend;LDA_03;global_subjectivity;abs_title_sentiment_polarity;componente_Guida,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.000000,-0.634367,0.000000,0.490012,0.000000,4.012188e-02,0.000000,0.000000,0.000000,-0.436924,7.694874,0.000000,-1.079270,0.000000,0.000000,0.000000,0.000000,NaN,0.00000,0.0,0.000000,3.805216,0.000000,0.000936,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,2.427357e+03,0.0,0.000000,0.0,0.0,NaN,0.0,0.000000,0.000000,0.000000,0.00000,60.0,0.0,0.057849,0.110851,0.0,0.000000,NaN,5.164272e-02,0.000000,0.000000,8.238564e+03,0.898223,0.087879,0.000000,0.0,0.0,0.0,0.0,shares^(1/3); ln(self_reference_avg_sharess + 1); (global_sbjectivity)^3,NaN,NaN,NaN,num_imgs;num_keywords;data_channel_is_entertainment;kw_avg_avg;self_reference_avg_sharess;weekday_is_saturday;LDA_03;global_subjectivity;abs_title_sentiment_polarity;componente_Guida,0.0,0.0,-1.206270e-16,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.894156e-02,0.000000,0.000000,0.000000,0.000000,9.877631,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.00000,0.0,0.000000,0.000000,0.000000,0.000946,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,2.513353e+03,0.0,0.000000,0.0,0.0,NaN,0.0,0.000000,0.000000,0.000000,0.00000,1.0,0.0,0.000000,0.000000,0.0,0.000000,NaN,3.913381e-02,0.000000,0.000000,8.228817e+03,0.924424,0.000000,0.000000,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,kw_avg_avg,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0
3,1574.540212,1571.959164,1573.657261,1573.063891,1573.758846,0.742015,0.000000,2.076378e+42,-0.081419,-1.134698,0.032512,0.000000,-1564.121053,-1.559277,-1.109111,-1.286187,-0.181903,-0.407369,-1.434486,NaN,-2.09004,0.0,-0.128028,3.139691,1.224501,0.001675,-0.000003,-0.000336,-0.000167,0.000000,0.0,-0.000333,0.0,0.0,1.667775e+23,0.0,-0.378349,0.0,0.0,NaN,0.0,-903304.174161,0.000047,0.025204,0.00000,33.0,0.0,0.026314,0.032515,0.0,0.016979,NaN,2.062671e+42,903304.285004,903304.720377,1.179295e+25,-1.000000,0.000000,0.000000,0.0,0.0,0.0,0.

In [368]:
bestPredictions = pd.read_csv('combinationPredictions.csv', index_col = 'id')
modelNumber = len(bestPredictions.columns)
bestPredictions[str(modelNumber)] = best_prediction.values
bestPredictions.to_csv('combinationPredictions.csv')
bestPredictions

,0,1,2,3,4
id,,,,,
5001,1358.148414,1141.555398,1632.090555,1272.049433,2401.761737
5002,2515.168041,2117.966847,2009.334676,2544.561856,3932.235427
5003,1828.820244,1421.507706,1615.060091,1791.053783,1864.323423
5004,2640.526357,1769.596154,1841.490925,2513.080476,3842.423709
5005,1905.230643,1622.917577,1893.579163,1977.919414,2914.786133
5006,2110.124375,1738.277910,1728.974722,1881.101016,1979.489084
5007,2524.392118,2512.576439,2111.549259,2825.419556,3690.771626
5008,2389.411519,2142.823901,2303.708603,3043.962317,3466.165086
5009,1723.644626,1727.557533,2199.327993,1817.618455,3390.749457
